In [1]:
__author__ = "amp"
__date__ = "2016_04_25"

%pylab inline
import pandas as pd
import os

from sklearn import linear_model
from sklearn.preprocessing import Imputer

#from ipynotifyer import notifyOnComplete as nf

Populating the interactive namespace from numpy and matplotlib


C:\Users\Alan\Anaconda\lib\site-packages\pandas\computation\__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
path = '../../../../Data/refined_data/puma50.csv'
puma = pd.read_csv(path, low_memory=False, index_col='st', na_values='.')
puma.shape

(5777, 1966)

In [3]:
yPath = '../../../../Data/refined_data/predict.csv'
ys = pd.read_csv(yPath, low_memory=False, index_col='st', na_values='.')[['internetP', 'label']]
ys.shape

(5777, 2)

In [4]:
def imputeDF(df):
    cols = df.columns
    L = df['label']
    
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    puma_imp = pd.DataFrame(imp.fit_transform(df.drop('label',1)))
    
    puma_imp.index = df.index
    puma_imp.columns = cols[:puma_imp.shape[1]]
    puma_imp['label'] = L
    print puma_imp.shape
    
    
    return puma_imp

    

In [5]:
puma_imp = imputeDF(puma)

(5777, 1966)


In [6]:
#Preprocessing

In [7]:
ip = ys['internetP']/100

In [8]:
#Splitting Data:

In [9]:
Xtest, Xtrain, Xval   = (x[1].drop('label',1) for x in puma_imp.groupby('label'))

In [10]:
ytest, ytrain, yval = (x[1]['internetP'] for x in ys.groupby('label'))

In [11]:
datas = {'Xtest':Xtest, 
         'Xtrain':Xtrain, 
         'Xval':Xval,
         'ytest':ytest,
         'ytrain':ytrain,
         'yval':yval}

In [12]:
#Setting Ridge Model

In [21]:
clf = linear_model.Ridge(alpha=0.5)

In [13]:
def evaluate(model, datas=datas ):
    try:
        x = model.fit(datas['Xtrain'], datas['ytrain'])
        print 'train:', x.score(datas['Xtrain'], datas['ytrain'])
        print 'valid:', x.score(datas['Xval'], datas['yval'])
    except Exception, e:
        print str(e)


In [23]:
evaluate(clf)

train: 0.957901755373
valid: 0.570544004682


In [24]:
#RidgeCV:

In [33]:
clf = linear_model.RidgeCV(alphas=(0.1,1.0,10.0))

TypeError: __init__() got an unexpected keyword argument 'positive'

In [29]:
evaluate(clf)

train: -1.37153984074e+21
valid: -4.65620897795e+20


In [30]:
#Lasso

In [31]:
clf = linear_model.Lasso(alpha = 0.5)

In [32]:
evaluate(clf)

train: 0.933581231571
valid: 0.834900582925


C:\Users\Alan\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:466: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations
  ConvergenceWarning)


In [34]:
#Elastic

In [35]:
clf = linear_model.ElasticNet(positive = True)

In [36]:
evaluate(clf)

train: 0.777523302635
valid: 0.757874732278


In [37]:
#ElasticCV

In [39]:
clf = linear_model.ElasticNetCV(positive = True)

In [40]:
evaluate(clf)

train: 0.0253132229713
valid: 0.0344018024086
